In [59]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from requests import post as requests_post
import wrds
import os
from statsmodels.sandbox.regression import gmm

In [2]:
# Import data

def request_wrds(path: str,
                 start_date: pd.Period,
                 end_date: pd.Period) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame):
    with wrds.Connection() as db:
        df_s12 = db.raw_sql(f'''
            SELECT *
            FROM tfn.s12
            WHERE fdate >= '{start_date}' AND fdate <= '{end_date}'
        ''', date_cols=['fdate'])
        df_s12.to_csv(f'{path}s12.csv')

        df_s34 = db.raw_sql(f'''
            SELECT *
            FROM tfn.s34
            WHERE fdate >= '{start_date}' AND fdate <= '{end_date}'
        ''', date_cols=['fdate'])
        df_s34.to_csv(f'{path}s34.csv')

        df_security = db.raw_sql(f'''
            SELECT *
            FROM ff.factors_monthly
            WHERE date >= '{start_date}' AND date <= '{end_date}'
        ''', date_cols=['date'])
        df_security.to_csv(f'{path}security.csv')

        return df_s12, df_s34, df_security


def load_wrds(path: str,
              start_date: pd.Period,
              end_date: pd.Period
              ) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):
    try:
        df_s12 = pd.read_csv(f'{path}s12.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_s12(df_s12)

        df_s12type5 = pd.read_csv(f'{path}s12type5.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_s12type5(df_s12type5)

        df_s34 = pd.read_csv(f'{path}s34.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_s34(df_s34)

        df_beta = pd.read_csv(f'{path}beta.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_beta(df_beta)

        df_security = pd.read_csv(f'{path}security.csv', encoding='ISO-8859-1', low_memory=False)
        log_import_security(df_security)

        return df_s12, df_s12type5, df_s34, df_beta, df_security
    except FileNotFoundError:
        return request_wrds(path, start_date, end_date)


def clean_imports(df_s12,
                  df_s12type5,
                  df_s34,
                  df_beta,
                  df_security,
                  start_date,
                  end_date
                  ) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):

    df_s12_clean = clean_s12(df_s12, start_date, end_date)
    log_clean_s12(df_s12_clean)

    df_s12type5_clean = clean_s12type5(df_s12type5, start_date, end_date)
    log_clean_s12type5(df_s12type5_clean)

    df_s34_clean = clean_s34(df_s34, start_date, end_date)
    log_clean_s34(df_s34_clean)

    df_beta_clean = clean_beta(df_beta, start_date, end_date)
    log_clean_beta(df_beta_clean)

    df_security_clean = clean_security(df_security, start_date, end_date)
    log_clean_security(df_security_clean)

    return df_s12_clean, df_s12type5_clean, df_s34_clean, df_beta_clean, df_security_clean


def clean_s12(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns = [
        'fundno',
        'rdate',
        'cusip',
        'shares'
    ]
    df['rdate'] = df['rdate'].apply(fix_date, args=(start_date, end_date))
    return (df[columns]
            .dropna(how='any', subset=['fundno', 'shares', 'cusip'])
            .drop_duplicates(subset=['fundno', 'rdate', 'cusip'], keep='last')
            .rename(columns={
                'rdate': 'date',
                'fundno': 'inv_id',
                'cusip': 'asset_id'
            })
            .set_index(['inv_id', 'date', 'asset_id']))


def clean_s12type5(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    return (df
            .assign(fdate=df['fdate'].apply(fix_date, args=(start_date, end_date)))
            .dropna(how='any', subset=['fundno', 'fdate'])
            .rename(columns={'fdate': 'date', 'fundno': 'inv_id'})
            .set_index(['inv_id', 'date']))


def clean_s34(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns = [
        'mgrno',
        'rdate',
        'typecode',
        'cusip',
        'shares',
        'backup_holding',
        'backup_me'
    ]
    df['rdate'] = df['rdate'].apply(fix_date, args=(start_date, end_date))
    df['backup_holding'] = df['shares'] * df['prc']
    df['backup_me'] = df['shrout2'] * df['prc'] * 1000
    return (df[columns]
            .dropna(how='any', subset=['cusip', 'shares'])
            .drop_duplicates(subset=['mgrno', 'rdate', 'cusip'])
            .rename(columns={
                'rdate': 'date',
                'mgrno': 'inv_id',
                'cusip': 'asset_id'
            })
            .set_index(['inv_id', 'date', 'asset_id']))


def clean_beta(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns=[
        'PERMNO',
        'DATE',
        'b_mkt',
        'b_smb',
        'b_hml'
    ]
    df['DATE'] = df['DATE'].apply(fix_date, args=(start_date, end_date)) + 6
    return (df[columns]
            .rename(columns={
                'DATE': 'date',
                'PERMNO':'permno'})
            .dropna()
            .drop_duplicates(subset=['date', 'permno'], keep='last')
            .set_index(['date', 'permno']))


def clean_security(df: pd.DataFrame, start_date: pd.Period, end_date: pd.Period) -> pd.DataFrame:
    columns = [
        'LPERMNO',
        'cusip',
        'datadate',
        'prccm',
        'cshoq'
    ]
    return (df[columns]
            .rename(columns={
                'LPERMNO': 'permno',
                'cusip': 'asset_id',
                'prccm': 'prc',
                'cshoq': 'shrout',
                'datadate': 'date'})
            .assign(
                shrout=df['cshoq'] * 1000000,
                asset_id=df['cusip'].apply(lambda  x: x[:-1]),
                date=df['datadate'].apply(fix_date, args=(start_date, end_date)))
            .dropna(how='any', subset=['asset_id', 'prc', 'shrout'])
            .drop_duplicates(subset=['date', 'permno'], keep='last')
            .set_index(['date', 'permno']))


def fix_date(date: str, start_date: pd.Period, end_date: pd.Period) -> pd.Period:
    date_converted = pd.Period(date, freq='M')

    if date_converted < start_date or date_converted > end_date:
        return np.NaN
    else:
        return date_converted

In [3]:
# Stocks Monthly

def merge_assets_factors(df_assets: pd.DataFrame, df_factors: pd.DataFrame) -> pd.DataFrame:
    df_merged = pd.merge(left=df_assets,
                      right=df_factors,
                      how='inner',
                      left_index=True,
                      right_index=True)
    df_merged_indexed = (df_merged
                         .reset_index()
                         .assign(date=lambda x: x['date'].astype(pd.PeriodDtype('Q')))
                         .drop_duplicates(subset=['date', 'asset_id'], keep='last')
                         .set_index(['date', 'asset_id']))
    log_asset_merge(df_merged_indexed)
    return df_merged_indexed


# Manager / Holdings

def match_fund_manager(df_fund: pd.DataFrame, df_manager: pd.DataFrame, df_key: pd.DataFrame) -> pd.DataFrame:
    # TODO
    # df_fund_wkey = df_fund.assign(mgrno=lambda x: df_key.loc[x.index.get_level_values(0), 'mgrcocd'])
    # df_merged = df_manager.merge(df_fund_wkey, how='outer', on='mgrno')
    
    df_fund_manager = (df_manager
                       .reset_index()
                       .assign(date=lambda x: x['date'].astype(pd.PeriodDtype('Q')))
                       .drop_duplicates(subset=['inv_id', 'date', 'asset_id'], keep='last')
                       .set_index(['inv_id', 'date', 'asset_id']))
    log_holding_merge(df_fund_manager)
    return df_fund_manager


def merge_holding_factor(df_holding: pd.DataFrame, df_asset: pd.DataFrame) -> pd.DataFrame:
    df_merged = pd.merge(
        left=df_holding,
        right=df_asset,
        how='left',
        left_index=True,
        right_index=True,
        indicator=True
    )

    df_merged = (df_merged
                 .reset_index()
                 .assign(
                   ccm_holding=lambda x: x['prc'] * x['shares'],
                   ccm_me=lambda x: x['prc'] * x['shrout'],
                   holding=lambda x: x['ccm_holding'].fillna(x['backup_holding']),
                   me=lambda x: x['ccm_me'].fillna(x['backup_me']),
                   type_code=lambda x: x['typecode'].fillna(0))
                 .astype({'inv_id': 'str', 'type_code': 'int'})
                 .drop(columns=['ccm_holding', 'backup_holding', 'ccm_me', 'backup_me'])
                 .dropna(subset=['holding'])
                 .set_index(['inv_id', 'date', 'asset_id'])
                 .sort_index())
    log_holding_factor_merge(df_merged)
    return df_merged


def create_outside_asset(df_holding_factor: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    df_inside = df_holding_factor[df_holding_factor['_merge'] == 'both'].drop(columns='_merge')
    df_outside = df_holding_factor[df_holding_factor['_merge'] == 'left_only'].groupby(['inv_id', 'date']).agg({
        'typecode': 'first',
        'holding': 'sum'
    }).assign(asset_id='-1').set_index('asset_id', append=True)
    log_outside_asset(df_outside)
    return df_inside, df_outside


def assetid_byinv(df_holding: pd.DataFrame) -> pd.Series:
    return df_holding.reset_index('asset_id')['asset_id']


def create_household_sector(df_outside: pd.DataFrame) -> pd.DataFrame:
    df_household = (df_outside
                      .groupby(['date', 'asset_id'])
                      .agg({
                        'shares': 'sum',
                        'prc': 'last',
                        'shrout': 'last',
                        'b_mkt': 'last',
                        'b_smb': 'last',
                        'b_hml': 'last',
                        'holding': 'sum',
                        'me': 'last'})
                      .assign(
                        shares=lambda x: x['shrout'] - x['shares'],
                        holding=lambda x: x['me'] - x['holding'],
                        inv_id='-1',
                        typecode=-1)
                    .set_index('inv_id', append=True)
                    .reorder_levels(['inv_id', 'date', 'asset_id']))
    log_household_sector(df_household)
    return df_household

In [51]:
# Data

def calc_inv_aum(df_inside: pd.DataFrame, df_outside: pd.DataFrame) -> pd.DataFrame:
    df_in_aum = (df_inside
                 .groupby(['inv_id', 'date'])
                 .agg({
                   'holding': 'sum',
                   'shares': 'count',
                   'typecode': 'last'})
                 .rename(columns={
                   'holding': 'in_aum',
                   'shares': 'n_holding'}))

    df_out_aum = (df_outside
                  .groupby(['inv_id', 'date'])
                  .agg({
                    'holding': 'sum',
                    'typecode': 'last'})
                  .rename(columns={'holding': 'out_aum'}))

    df_merged = pd.merge(
        left=df_in_aum,
        right=df_out_aum,
        how='outer',
        left_index=True,
        right_index=True)
    
    df_inv_aum = (df_merged
                  .assign(
                    typecode=lambda x: x['typecode_x'].fillna(x['typecode_y']),
                    n_holding=lambda x: x['n_holding'].fillna(0),
                    out_aum=lambda x: x['out_aum'].fillna(0),
                    in_aum=lambda x: x['in_aum'].fillna(0),
                    aum=lambda x: x['out_aum'] + x['in_aum'])
                  .drop(columns=['typecode_x', 'typecode_y']))

    log_inv_aum(df_inv_aum)
    return df_inv_aum


def bin_concentrated_inv(df_inside: pd.DataFrame, df_inv_aum: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    cutoff = 500
    household_mask = (df_inv_aum['out_aum'] == 0) | (df_inv_aum['in_aum'] == 0)
    df_valid = df_inv_aum[~household_mask]
    
    diversified_mask = df_valid['n_holding'] >= cutoff
    df_aum_diversified = df_valid[diversified_mask]
    df_aum_concentrated = df_valid[~diversified_mask].assign(bin=np.nan)
    
    arr_dates = df_aum_concentrated.index.get_level_values('date')
    arr_unique_dates = arr_dates.unique()
    arr_typecodes = df_aum_concentrated['typecode'].unique()
    
    for date in arr_unique_dates:
        for typecode in arr_typecodes:
            mask = (arr_dates == date) & (df_aum_concentrated['typecode'] == typecode)
            df_type_date = df_aum_concentrated[mask]
            n_bins = np.ceil(df_type_date['n_holding'].sum() / (2 * cutoff)).astype(int)
            if n_bins <= 1:
                df_aum_concentrated.loc[mask, 'bin'] = 0
            else:
                df_aum_concentrated.loc[mask, 'bin'] = pd.qcut(x=df_type_date['aum'], q=n_bins, labels=False).astype(int)

    df_aum_concentrated = (df_aum_concentrated
                           .astype({'bin':'str'})
                           .assign(bin=lambda x: x['typecode'].astype('str') + ':' + x['bin']))
    df_concentrated_binned = (df_aum_concentrated
                              .reset_index()
                              .groupby(['bin', 'date'])
                              .agg({
                                'in_aum': 'sum',
                                'out_aum': 'sum',
                                'aum': 'sum',
                                'n_holding': 'sum',
                                'typecode': 'last'})
                              .rename_axis(index={'bin': 'inv_id'}))
    df_aum_binned = (pd.concat([df_aum_diversified, df_concentrated_binned])
                     .sort_index()
                     .assign(
                       out_weight=lambda x: x['out_aum'] / x['aum'],
                       in_weight=lambda x: x['in_aum'] / x['aum']))

    df_inside_merged = pd.merge(
        left=df_inside,
        right=df_aum_concentrated['bin'],
        how='left',
        left_index=True,
        right_index=True)
    df_inside_binned = (df_inside_merged
                        .reset_index()
                        .assign(bin=lambda x: x['bin'].fillna(x['inv_id']))
                        .groupby(['bin', 'date', 'asset_id'])
                        .agg({
                          'holding': 'sum',
                          'prc': 'last',
                          'shrout': 'last',
                          'me': 'last',
                          'b_mkt': 'last',
                          'b_smb': 'last',
                          'b_hml': 'last'})
                        .rename_axis(index={'bin': 'inv_id'}))
    
    log_bins(df_inside_binned, df_aum_binned)
    return df_inside_binned, df_aum_binned


def calc_inv_universe(df_holding: pd.DataFrame) -> pd.DataFrame:
    num_quarters = 11
    df_assetid_byinv = assetid_byinv(df_holding).sort_index()
    idx_inv_universe = df_assetid_byinv.index.unique()
    df_inv_universe = pd.DataFrame(index=idx_inv_universe, columns=['inv_universe'])

    def calc_past_quarters(i: int, d: pd.Period) -> np.array:
        prev_date = d - num_quarters
        asset_id_within_range = df_assetid_byinv.loc[i].loc[prev_date:d]
        inv_uni = asset_id_within_range.unique().tolist()
        return inv_uni

    for (inv_id, date) in idx_inv_universe.to_flat_index():
        inv_uni = calc_past_quarters(inv_id, date)
        df_inv_universe.loc[(inv_id, date), 'inv_universe'] = inv_uni

    df_inv_universe = df_inv_universe.assign(uni_size=lambda x: x['inv_universe'].apply(len))
    log_inv_universe(df_inv_universe)
    return df_inv_universe


def create_equal_allocation(df_inv_universe: pd.DataFrame, df_aum_binned: pd.DataFrame) -> pd.DataFrame:
    df_equal_alloc = pd.merge(
        left=df_inv_universe,
        right=df_aum_binned,
        how='inner',
        left_index=True,
        right_index=True)
    
    return (df_equal_alloc
            .assign(allocation=lambda x: x['aum'] / (x['uni_size'] + 1))
            .explode('inv_universe')
            .rename(columns={'inv_universe':'asset_id'})
            .set_index('asset_id', append=True))


def create_total_allocation(df: pd.DataFrame) -> pd.DataFrame:
    return (df['allocation']
            .groupby('asset_id')
            .sum())


def create_instrument(df_inv_universe: pd.DataFrame, df_aum_binned: pd.DataFrame) -> pd.DataFrame:
    df_equal_allocation = create_equal_allocation(df_inv_universe, df_aum_binned)
    total_allocation = create_total_allocation(df_equal_allocation)
    df_instrument = df_equal_allocation.assign(iv_me=lambda x: total_allocation - x['allocation'])
    log_instrument(df_instrument)
    return df_instrument

In [84]:
# Estimation

def calc_holding_weights(df_instrument: pd.DataFrame, df_inside_binned: pd.DataFrame, df_asset: pd.DataFrame) -> pd.DataFrame:
    df_merged = pd.merge(
        left=df_instrument,
        right=df_inside_binned,
        how='outer',
        left_index=True,
        right_index=True)
    
    min_holding = 500
    mask = (df_merged['n_holding'] >= min_holding) & (df_merged['out_weight'] > 0) & (df_merged['in_weight'] > 0) & (df_merged['me'] > 0) & (df_merged['iv_me'] > 0)

    df_weights = (df_merged.loc[mask]
                .assign(
                    ln_me=lambda x: np.log(x['me']),
                    ln_iv_me=lambda x: np.log(x['iv_me']),
                    weight=lambda x: x['holding'] / x['aum'],
                    ln_weight=lambda x: np.log(x['weight']),
                    rweight=lambda x: x['weight'] / x['out_weight'],
                    ln_rweight=lambda x: np.log(x['rweight']),
                    cons=lambda x: x['ln_rweight'].groupby(['inv_id', 'date']).transform('mean'))
                .reset_index('asset_id'))
    
    log_holding_weights(df_weights)
    return df_weights


def momcond(params, exog):
    bound = 0.999
    exog = exog.T
    
    beta_ln_me = params[0]
    beta_characteristics = params[1:]
    
    ln_me = exog[0]
    rweight = exog[1]
    characteristics = exog[2:]
    
    ln_me_term = (bound - np.exp(-1 * beta_ln_me)) * ln_me
    characteristics_term = np.dot(beta_characteristics, characteristics)
    pred_weight = np.exp(-1 * (ln_me_term + characteristics_term))
    
    return rweight * pred_weight


def unpack_result(result):
    params = result.params
    return [params[0], params[1], params[2], params[3], params[4]]


def estimate_model(df_weights: pd.DataFrame):
    idx = df_weights.index.unique()
    cols = ['beta_ln_me', 'beta_cons', 'beta_b_mkt', 'beta_b_smb', 'beta_b_hml']
    df_model = pd.DataFrame(
        index=idx,
        columns=cols)

    for (inv_id, date) in idx.to_flat_index():
        print(inv_id, date)
        data = df_weights.loc[(inv_id, date)]
        exog = np.asarray(data[['ln_me', 'rweight', 'cons', 'b_mkt', 'b_smb', 'b_hml']])
        instrument = np.asarray(data[['ln_iv_me', 'rweight', 'cons', 'b_mkt', 'b_smb', 'b_hml']])
        n = exog.shape[0]
        endog = np.ones(n)
        
        try:
            model = gmm.NonlinearIVGMM(
                endog=endog, 
                exog=exog, 
                instrument=instrument, 
                func=momcond, 
                k_moms=1)
            w0inv = np.dot(instrument.T, instrument) / n
            start_params = np.zeros(len(cols))
            result = model.fit(
                start_params=start_params,
                maxiter=2,
                inv_weights=w0inv)
            log_results(result, cols)
            df_model.loc[(inv_id, date)] = unpack_result(result)
        except:
            continue
    
    log_params(df_model)
    return df_model


def calc_latent_demand(df_model: pd.DataFrame, df_weights: pd.DataFrame) -> pd.DataFrame:
    bound = 0.999
    df_model = df_model.assign(beta_ln_me=lambda x: (bound - np.exp(-1 * x['beta_ln_me'])))
    
    arr_param = df_model[['beta_ln_me', 'beta_cons', 'beta_b_mkt', 'beta_b_smb', 'beta_b_hml']].to_numpy()
    arr_char = df_weights[['ln_me', 'cons', 'b_mkt', 'b_smb', 'b_hml']].to_numpy().T
    df_results = df_model.assign(
        pred_rweight=arr_param @ arr_char,
        latent_demand=lambda x: x['pred_rweight'] * x['rweight'])
    return df_results

In [6]:
# Log

def log_import_s12(df_s12: pd.DataFrame):
    dateindex_s12 = df_s12['rdate']
    print('Imported s12')
    print('Number of holdings:  ', len(df_s12))
    print('Earliest date:  ', min(dateindex_s12))
    print('Latest date:  ', max(dateindex_s12))


def log_import_s12type5(df_s12type5: pd.DataFrame):
    dateindex_s12type5 = df_s12type5['fdate']
    print()
    print('Imported s12type5')
    print('Number of holdings:  ', len(df_s12type5))
    print('Earliest date:  ', min(dateindex_s12type5))
    print('Latest date:  ', max(dateindex_s12type5))


def log_import_s34(df_s34: pd.DataFrame):
    dateindex_s34 = df_s34['rdate']
    print()
    print('Imported s34')
    print('Number of holdings:  ', len(df_s34))
    print('Earliest date:  ', min(dateindex_s34))
    print('Latest date:  ', max(dateindex_s34))


def log_import_beta(df_beta: pd.DataFrame):
    dateindex_ffm = df_beta['DATE']
    print()
    print('Imported betas')
    print('Number of dates:  ', len(df_beta))
    print('Earliest date:  ', min(dateindex_ffm))
    print('Latest date:  ', max(dateindex_ffm))


def log_import_security(df_security: pd.DataFrame):
    dateindex_security = df_security['datadate']
    print()
    print('Imported security')
    print('Number of holdings:  ', len(df_security))
    print('Earliest date:  ', min(dateindex_security))
    print('Latest date:  ', max(dateindex_security))


def log_clean_s12(df_s12_clean: pd.DataFrame):
    dateindex_s12 = df_s12_clean.index.get_level_values('date')
    print('Cleaned s12')
    print('Number of holdings:  ', len(df_s12_clean))
    print('Earliest date:  ', min(dateindex_s12))
    print('Latest date:  ', max(dateindex_s12))


def log_clean_s12type5(df_s12type5_clean: pd.DataFrame):
    dateindex_s12type5 = df_s12type5_clean.index.get_level_values('date')
    print()
    print('Cleaned s12type5')
    print('Number of firm/dates:  ', len(df_s12type5_clean))
    print('Earliest date:  ', min(dateindex_s12type5))
    print('Latest date:  ', max(dateindex_s12type5))


def log_clean_s34(df_s34_clean: pd.DataFrame):
    dateindex_s34 = df_s34_clean.index.get_level_values('date')
    print()
    print('Cleaned s34')
    print('Number of holdings:  ', len(df_s34_clean))
    print('Earliest date:  ', min(dateindex_s34))
    print('Latest date:  ', max(dateindex_s34))


def log_clean_beta(df_beta_clean: pd.DataFrame):
    dateindex_beta = df_beta_clean.index.get_level_values('date')
    print()
    print('Cleaned beta')
    print('Number of dates:  ', len(df_beta_clean))
    print('Earliest date:  ', min(dateindex_beta))
    print('Latest date:  ', max(dateindex_beta))


def log_clean_security(df_security_clean: pd.DataFrame):
    dateindex_security = df_security_clean.index.get_level_values('date')
    print()
    print('Cleaned security')
    print('Number of asset/dates:  ', len(df_security_clean))
    print('Earliest date:  ', min(dateindex_security))
    print('Latest date:  ', max(dateindex_security))


def log_holding_merge(df_merged: pd.DataFrame):
    print('Merged s12 and s34')
    print('Number of holdings:  ', len(df_merged))
    print()


def log_asset_merge(df_merged: pd.DataFrame):
    print('Merged assets and factors')
    print('Number of assets/dates:  ', len(df_merged))
    print()


def log_holding_factor_merge(df_merged: pd.DataFrame):
    print('Merged holdings and factors')
    print('Number of assets/dates:  ', len(df_merged))
    print()


def log_household_sector(df_household: pd.DataFrame):
    print('Created household sector')
    print('Number of holdings:  ', len(df_household))


def log_outside_asset(df_holding: pd.DataFrame):
    print('Created outside asset')
    print('Number of holdings:  ', len(df_holding))


def log_inv_aum(df_inv_aum: pd.DataFrame):
    print('Calculated investor AUM')
    print(df_inv_aum.describe())


def log_bins(df_inside_binned: pd.DataFrame, df_aum_binned: pd.DataFrame):
    print('Binned investors')
    print('Number of investors by aum:  ', len(df_inside_binned.index.unique('inv_id')))
    print('Number of investors by holding:  ', len(df_aum_binned.index.unique('inv_id')))


def log_inv_universe(df_inv_uni: pd.DataFrame):
    print('Created investment universe')
    print(df_inv_uni.describe())
    
    
def create_tables(df_instrument: pd.DataFrame, df_inv_uni: pd.DataFrame):
    df_pctile = df_inv_uni.assign(pctile=0)
    arr_dates = df_pctile.index.get_level_values('date').unique()
    
    for date in arr_dates:
        data = df_pctile.loc[date, 'aum']
        df_pctile.loc[date, 'pctile'] = pd.qcut(data, q=100)
        
    df_grouped_pctile = (df_pctile['uni_persistence']
                         .groupby(['pctile'])
                         .median())
    print(df_grouped_pctile.head(10))


def log_instrument(df_instrument: pd.DataFrame):
    print('Created market equity instrument')
    print(df_instrument.describe())
    print()
    

def log_holding_weights(df_model: pd.DataFrame):
    print('Calculated holding weights')
    print(df_model.describe())


def log_results(result, cols):
    # print(result.summary(yname='Latent demand', xname=cols))
    print()
    

def log_params(df_params: pd.DataFrame):
    print('Estimated parameters')
    print()

    
def j_test(df_results: pd.DataFrame):
    # TODO
    pass


def graph_index_params(df_results: pd.DataFrame):
    # TODO
    pass


def graph_type_params(df_results: pd.DataFrame):
    df_types = df_results.groupby(['type', 'date'], as_index=False).mean()
    plt.plot(df_types, x='date', y='beta_ln_me', hue='type')
    plt.show()

In [8]:
# Main

path = 'data/'
output = 'output/'
start_date = pd.Period('2012-01', freq='M')
end_date = pd.Period('2017-12', freq='M')

In [8]:
print('\n---------------Starting Imports---------------------------\n')
dfs = load_wrds(path, start_date, end_date)

print('\n---------------Starting Cleaning---------------------------\n')
df_s12_clean, df_s12type5_clean, df_s34_clean, df_beta_clean, df_security_clean = clean_imports(
    *dfs,
    start_date,
    end_date
)

df_s12_clean.to_csv(os.path.join(output, 'df_s12_clean.csv'))
df_s12type5_clean.to_csv(os.path.join(output, 'df_s12type5_clean.csv'))
df_s34_clean.to_csv(os.path.join(output, 'df_s34_clean.csv'))
df_beta_clean.to_csv(os.path.join(output, 'df_beta_clean.csv'))
df_security_clean.to_csv(os.path.join(output, 'df_security_clean.csv'))


---------------Starting Imports---------------------------

Imported s12
Number of holdings:   30987
Earliest date:   2014-06-30
Latest date:   2017-12-31

Imported s12type5
Number of holdings:   572518
Earliest date:   12/31/1994
Latest date:   9/30/2022

Imported s34
Number of holdings:   22707709
Earliest date:   2012-03-31
Latest date:   2017-12-31

Imported betas
Number of dates:   432458
Earliest date:   2012-01-31
Latest date:   2017-12-29

Imported security
Number of holdings:   402452
Earliest date:   2012-01-31
Latest date:   2017-12-31

---------------Starting Cleaning---------------------------
Cleaned s12
Number of holdings:   27327
Earliest date:   2014-06
Latest date:   2017-12

Cleaned s12type5
Number of firm/dates:   49372
Earliest date:   2012-03
Latest date:   2017-12

Cleaned s34
Number of holdings:   22267070
Earliest date:   2012-03
Latest date:   2017-12

Cleaned beta
Number of dates:   432414
Earliest date:   2012-07
Latest date:   2018-06

Cleaned security
Num

In [9]:
print('\n---------------Merging Assets/Factors---------------------------\n')
df_asset = merge_assets_factors(df_security_clean, df_beta_clean)
df_asset.to_csv(os.path.join(output, 'df_asset.csv'))

print('\n---------------Merging s12/s34 Holdings---------------------------\n')
df_fund_manager = match_fund_manager(df_s12_clean, df_s34_clean, df_s12type5_clean)
df_fund_manager.to_csv(os.path.join(output, 'df_fund_manager.csv'))


---------------Merging Assets/Factors---------------------------
Merged assets and factors
Number of assets/dates:   98918

---------------Merging s12/s34 Holdings---------------------------
Merged s12 and s34
Number of holdings:   22267070


In [10]:
print('\n---------------Merging Holdings/Factors---------------------------\n')
df_holding_factor = merge_holding_factor(df_fund_manager, df_asset)
df_holding_factor.to_csv(os.path.join(output, 'df_holding_factor.csv'))

print('\n---------------Partitioning Outside Asset---------------------------\n')
df_inside, df_outside = create_outside_asset(df_holding_factor)
df_inside.to_csv(os.path.join(output, 'df_inside.csv'))
df_outside.to_csv(os.path.join(output, 'df_outside.csv'))

print('\n---------------Creating Household Sector---------------------------\n')
df_household = create_household_sector(df_inside)
df_household.to_csv(os.path.join(output, 'df_household.csv'))

print('\n---------------Calculating Investor AUM---------------------------\n')
df_inv_aum = calc_inv_aum(df_inside, df_outside)
df_inv_aum.to_csv(os.path.join(output, 'df_inv_aum.csv'))


---------------Merging Holdings/Factors---------------------------
Merged holdings and factors
Number of assets/dates:   22243229

---------------Partitioning Outside Asset---------------------------
Created outside asset
Number of holdings:   90936

---------------Creating Household Sector---------------------------
Created household sector
Number of holdings:   75467

---------------Calculating Investor AUM---------------------------
Calculated investor AUM
             in_aum     n_holding       out_aum      typecode           aum
count  9.282700e+04  92827.000000  9.282700e+04  92824.000000  9.282700e+04
mean   3.520373e+09    163.105831  1.046345e+09      4.522149  4.566718e+09
std    2.915269e+10    351.706418  8.350515e+09      0.764381  3.438445e+10
min    0.000000e+00      0.000000  0.000000e+00      1.000000  1.552000e+01
25%    6.046979e+07     13.000000  3.739851e+07      4.000000  1.548055e+08
50%    1.947032e+08     52.000000  1.128732e+08      5.000000  3.607226e+08
75%

In [48]:
df_inside = pd.read_csv(os.path.join(output, 'df_inside.csv'), dtype={'date':pd.PeriodDtype('Q')})
df_inv_aum = pd.read_csv(os.path.join(output, 'df_inv_aum.csv'), parse_dates=['date'], date_format='YQ')
df_outside = pd.read_csv(os.path.join(output, 'df_outside.csv'), parse_dates=['date'], date_format='YQ')

df_inv_aum['date'] = df_inv_aum['date'].astype(pd.PeriodDtype(freq='Q'))
df_outside['date'] = df_outside['date'].astype(pd.PeriodDtype('Q'))

df_inside.set_index(['inv_id', 'date', 'asset_id'], inplace=True)
df_inv_aum.set_index(['inv_id', 'date'], inplace=True)
df_outside.set_index(['inv_id', 'date', 'asset_id'], inplace=True)

In [56]:
df_asset = pd.read_csv(os.path.join(output, 'df_asset.csv'), parse_dates=['date'], date_format='YQ')
df_asset.set_index(['date', 'asset_id'], inplace=True)

In [49]:
print('\n---------------Pooling Investors By Type/Size---------------------------\n')
df_inside_binned, df_aum_binned = bin_concentrated_inv(df_inside, df_inv_aum)
# df_inside_binned.to_csv(os.path.join(output, 'df_inside_binned.csv'))
# df_aum_binned.to_csv(os.path.join(output, 'df_aum_binned.csv'))


---------------Pooling Investors By Type/Size---------------------------
Binned investors
Number of investors by aum:   1512
Number of investors by holding:   1022


In [52]:
print('\n---------------Tracking Investment Universe---------------------------\n')
df_inv_universe = calc_inv_universe(df_inside_binned)
# df_inv_universe.to_csv(os.path.join(output, 'df_inv_universe.csv'))

print('\n---------------Calculating Instrument---------------------------\n')
df_instrument = create_instrument(df_inv_universe, df_aum_binned)
# df_instrument.to_csv(os.path.join(output, 'df_instrument.csv'))


---------------Tracking Investment Universe---------------------------
Created investment universe
           uni_size
count  16151.000000
mean    1445.355829
std      863.225923
min        1.000000
25%      817.000000
50%     1463.000000
75%     2063.000000
max     4973.000000

---------------Calculating Instrument---------------------------
Created market equity instrument
           uni_size        in_aum     n_holding       out_aum      typecode  \
count  2.333144e+07  2.333144e+07  2.333144e+07  2.333144e+07  2.333126e+07   
mean   1.961826e+03  3.184937e+10  1.238825e+03  7.194824e+09  4.412841e+00   
std    7.345614e+02  1.038051e+11  6.655928e+02  1.860047e+10  1.000173e+00   
min    3.600000e+01  1.422871e+07  4.100000e+01  1.588244e+06  1.000000e+00   
25%    1.450000e+03  1.498401e+09  7.330000e+02  6.731325e+08  4.000000e+00   
50%    1.961000e+03  4.782553e+09  1.056000e+03  1.716648e+09  5.000000e+00   
75%    2.408000e+03  2.023541e+10  1.606000e+03  5.105955e+09  5.000

In [72]:
print('\n---------------Calculating Holding Weights---------------------------\n')
df_weights = calc_holding_weights(df_instrument, df_inside_binned, df_asset)
# df_weights.to_csv(os.path.join(output, 'df_weights.csv'))


---------------Calculating Holding Weights---------------------------


C:\Users\wyatt\AppData\Local\Temp\ipykernel_12972\3568120018.py:4: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  df_merged = pd.merge(


Calculated holding weights
           uni_size        in_aum     n_holding       out_aum      typecode  \
count  1.218451e+07  1.218451e+07  1.218451e+07  1.218451e+07  1.218451e+07   
mean   1.935696e+03  4.563090e+10  1.418160e+03  9.751939e+09  4.327332e+00   
std    8.411012e+02  1.278960e+11  7.075418e+02  2.246877e+10  1.127583e+00   
min    2.940000e+02  1.422871e+07  5.000000e+02  1.588244e+06  1.000000e+00   
25%    1.261000e+03  2.106718e+09  8.220000e+02  7.709783e+08  4.000000e+00   
50%    1.900000e+03  8.366262e+09  1.242000e+03  2.361984e+09  5.000000e+00   
75%    2.478000e+03  3.344969e+10  1.914000e+03  7.248827e+09  5.000000e+00   
max    4.973000e+03  1.587447e+12  4.090000e+03  2.292610e+11  5.000000e+00   

                aum    out_weight     in_weight    allocation         iv_me  \
count  1.218451e+07  1.218451e+07  1.218451e+07  1.218451e+07  1.218451e+07   
mean   5.538284e+10  2.503483e-01  7.496517e-01  2.412544e+07  1.465617e+11   
std    1.476544e+11  1.5

In [ ]:
print('\n---------------Estimating Demand System---------------------------\n')
df_model = estimate_model(df_weights)
# TODO suppress print statements
# df_model.to_csv(os.path.join(output, 'df_model.csv'))


---------------Estimating Demand System---------------------------
185 2016Q4
Optimization terminated successfully.
         Current function value: 0.024654
         Iterations: 83
         Function evaluations: 101
         Gradient evaluations: 101
Optimization terminated successfully.
         Current function value: 0.012982
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12

185 2017Q1
Optimization terminated successfully.
         Current function value: 0.020856
         Iterations: 85
         Function evaluations: 104
         Gradient evaluations: 104
Optimization terminated successfully.
         Current function value: 0.018439
         Iterations: 8
         Function evaluations: 13
         Gradient evaluations: 13

185 2017Q2
Optimization terminated successfully.
         Current function value: 0.019692
         Iterations: 84
         Function evaluations: 103
         Gradient evaluations: 103
Optimization terminated successfu

C:\Users\wyatt\AppData\Local\Temp\ipykernel_12972\3317024509.py:42: RuntimeWarning: overflow encountered in exp
  pred_weight = np.exp(-1 * (ln_me_term + characteristics_term))
C:\Users\wyatt\anaconda3\Lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\wyatt\anaconda3\Lib\site-packages\statsmodels\tools\numdiff.py:157: RuntimeWarning: invalid value encountered in subtract
  grad[k, :] = (f(*((x+ei,)+args), **kwargs) -
C:\Users\wyatt\AppData\Local\Temp\ipykernel_12972\3317024509.py:40: RuntimeWarning: overflow encountered in exp
  ln_me_term = (bound - np.exp(-1 * beta_ln_me)) * ln_me
C:\Users\wyatt\AppData\Local\Temp\ipykernel_12972\3317024509.py:42: RuntimeWarning: overflow encountered in exp
  pred_weight = np.exp(-1 * (ln_me_term + characteristics_term))
C:\Users\wyatt\anaconda3\Lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered i

Optimization terminated successfully.
         Current function value: 0.247605
         Iterations: 66
         Function evaluations: 90
         Gradient evaluations: 90
Optimization terminated successfully.
         Current function value: 0.007918
         Iterations: 7
         Function evaluations: 11
         Gradient evaluations: 11

220 2013Q2
Optimization terminated successfully.
         Current function value: 0.255527
         Iterations: 68
         Function evaluations: 83
         Gradient evaluations: 83
Optimization terminated successfully.
         Current function value: 0.032569
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12

220 2013Q3
Optimization terminated successfully.
         Current function value: 0.215664
         Iterations: 71
         Function evaluations: 87
         Gradient evaluations: 87
Optimization terminated successfully.
         Current function value: 0.043469
         Iterations: 8
         Functi

C:\Users\wyatt\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:1360: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)
C:\Users\wyatt\AppData\Local\Temp\ipykernel_12972\3317024509.py:42: RuntimeWarning: overflow encountered in exp
  pred_weight = np.exp(-1 * (ln_me_term + characteristics_term))
C:\Users\wyatt\anaconda3\Lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\wyatt\anaconda3\Lib\site-packages\statsmodels\tools\numdiff.py:157: RuntimeWarning: invalid value encountered in subtract
  grad[k, :] = (f(*((x+ei,)+args), **kwargs) -
C:\Users\wyatt\AppData\Local\Temp\ipykernel_12972\3317024509.py:40: RuntimeWarning: overflow encountered in exp
  ln_me_term = (bound - np.exp(-1 * beta_ln_me)) * ln_me
C:\Users\wyatt\AppData\Local\Temp\ipykernel_12972\3317024509.py:42: RuntimeWar

         Current function value: 0.674886
         Iterations: 1000
         Function evaluations: 1239
         Gradient evaluations: 1239
         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112

5850 2012Q3
Optimization terminated successfully.
         Current function value: 0.397583
         Iterations: 39
         Function evaluations: 59
         Gradient evaluations: 59
Optimization terminated successfully.
         Current function value: 0.001231
         Iterations: 65
         Function evaluations: 80
         Gradient evaluations: 80

5850 2012Q4
Optimization terminated successfully.
         Current function value: 0.475094
         Iterations: 46
         Function evaluations: 62
         Gradient evaluations: 62
Optimization terminated successfully.
         Current function value: 0.001064
         Iterations: 83
         Function evaluations: 102
         Gradient evaluations: 102

5850 2013Q1
O

...

         Current function value: 0.246737
         Iterations: 1000
         Function evaluations: 1288
         Gradient evaluations: 1288
         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112


C:\Users\wyatt\anaconda3\Lib\site-packages\statsmodels\tools\numdiff.py:151: RuntimeWarning: invalid value encountered in subtract
  grad[k, :] = (f(*((x+ei,) + args), **kwargs) - f0)/epsilon[k]



13636 2015Q2
Optimization terminated successfully.
         Current function value: 0.592531
         Iterations: 40
         Function evaluations: 64
         Gradient evaluations: 64
Optimization terminated successfully.
         Current function value: 0.000146
         Iterations: 164
         Function evaluations: 214
         Gradient evaluations: 214

13636 2015Q3
Optimization terminated successfully.
         Current function value: 0.517613
         Iterations: 77
         Function evaluations: 104
         Gradient evaluations: 104
Optimization terminated successfully.
         Current function value: 0.000167
         Iterations: 171
         Function evaluations: 210
         Gradient evaluations: 210

13655 2012Q4
Optimization terminated successfully.
         Current function value: 0.625032
         Iterations: 44
         Function evaluations: 66
         Gradient evaluations: 66
Optimization terminated successfully.
         Current function value: 0.000182
         I

C:\Users\wyatt\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:1360: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


Optimization terminated successfully.
         Current function value: 0.000098
         Iterations: 18
         Function evaluations: 25
         Gradient evaluations: 25

13663 2014Q3
Optimization terminated successfully.
         Current function value: 0.157378
         Iterations: 23
         Function evaluations: 44
         Gradient evaluations: 44
Optimization terminated successfully.
         Current function value: 0.000124
         Iterations: 19
         Function evaluations: 26
         Gradient evaluations: 26

13663 2014Q4
Optimization terminated successfully.
         Current function value: 0.168422
         Iterations: 26
         Function evaluations: 49
         Gradient evaluations: 49
Optimization terminated successfully.
         Current function value: 0.000101
         Iterations: 22
         Function evaluations: 30
         Gradient evaluations: 30

13663 2015Q1
Optimization terminated successfully.
         Current function value: 0.210889
         Iteration

In [ ]:
print('\n---------------Calculating Latent Demand---------------------------\n')
df_results = calc_latent_demand(df_model, df_weights)
# df_results.to_csv(os.path.join(output, 'df_results.csv'))

In [ ]:
df_results

In [ ]:
graph_type_params(df_results)

print('\n---------------Finished---------------------------\n')